# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --no-build-isolation --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

[2024-02-08 02:29:39,446 88602:140704380690048][patches.py:9 todd <module>] INFO: `ipdb` is installed. Using it for debugging.
/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-08 02:29:41,509 88602:140704380690048][base.py:55 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7a4uee5t
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[dict(type='LogCallback', interval=5)],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-08 02:29:41,865 88602:140704380690048][base.py:55 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:41,872 88602:140704380690048][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:29:41,881 88602:140704380690048][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-08 02:29:41,891 88602:140704380690048][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-08 02:29:41,924 88602:140704380690048][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpr4b_9xdt
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:29:42,236 88602:140704380690048][base.py:55 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:42,242 88602:140704380690048][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([6, 8]), 'y': tensor([12, 16])} weight=0.000 loss=14.000
[2024-02-08 02:29:42,243 88602:140704380690048][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([ 1, 10]), 'y': tensor([ 2, 20])} weight=0.000 loss=11.000
[2024-02-08 02:29:42,245 88602:140704380690048][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [3/8] batch={'x': tensor([2, 3]), 'y': tensor([4, 6])} weight=0.000 loss=5.000
[2024-02-08 02:29:42,247 88602:140704380690048][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [4/8] batch={'x': tensor([5, 9]), 'y': tensor([10, 18])} weight=0.000 loss=14.000
[2024-0

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:29:42,266 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:42,268 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-02-08 02:29:42,270 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([6, 4]), 'y': tensor([12,  8])} weight=0.000 loss=10.000
[2024-02-08 02:29:42,272 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([7, 5]), 'y': tensor([14, 10])} weight=0.000 loss=12.000
[2024-02-08 02:29:42,274 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [3/15] batch={'x': tensor([ 1, 10]), 'y': tensor([ 2, 20])} weight=0.000 loss=11.000
[2024-02-08 02:29:42,277 88602:140704380690048][log.py:93 todd.E

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:29:42,435 88602:140704380690048][log.py:55 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: 315b3b2
git_status: 
M todd/runners/callbacks/composed.py
 M todd/utils/misc.py
 M todd/utils/torch.py
 M tutorials/runners.ipynb
[2024-02-08 02:29:42,437 88602:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:42,442 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:29:42,446 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 los

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-02-08 02:29:42,478 88602:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:42,484 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:29:42,488 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-08 02:29:42,492 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-08 02:29:42,496 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpqfpbk8sm
└── log_callback
    └── 2024-02-08T02-29-42_477193-08-00.log

2 directories, 1 file

[2024-02-08 02:29:42,478 88602:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:42,484 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:29:42,488 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-08 02:29:42,492 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-08 02:29:42,496 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-02-08 02:29:43,110 88602:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:43,649 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:29:44,167 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-08 02:29:44,687 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-08 02:29:45,208 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", decay=0.2),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-02-08 02:29:45,223 88602:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:29:46,740 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:03 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:29:50,759 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:04 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-08 02:29:55,780 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-08 02:30:00,796 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:30:00,948 88602:140704380690048][log.py:55 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: 315b3b2
git_status: 
M todd/runners/callbacks/composed.py
 M todd/utils/misc.py
 M todd/utils/torch.py
 M tutorials/runners.ipynb
[2024-02-08 02:30:00,950 88602:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:00,955 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-08 02:30:00,958 88602:140704380690048][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:00 batch={'x': tensor([10]), 'y': tensor

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-02-08 02:30:01,045 88602:140704380690048][git.py:41 todd.Validator.git_callback init] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpfz__u5vg/git_callback/git_diff_2024-02-08T02-30-01_045098-08-00.log
[2024-02-08 02:30:01,048 88602:140704380690048][base.py:55 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



diff --git a/todd/runners/callbacks/composed.py b/todd/runners/callbacks/composed.py
index 772fb59..cb5285a 100644
--- a/todd/runners/callbacks/composed.py
+++ b/todd/runners/callbacks/composed.py
@@ -2,139 +2,109 @@ __all__ = [
     'ComposedCallback',
 ]
 
-from collections import UserList
-from typing import Any, Iterable, Literal, Mapping, TypedDict
+from typing import Any, Iterable, Literal, Mapping
+
+from ...utils import PriorityQueue
 
 from ...base import CallbackRegistry, Config
 from .base import BaseCallback
 
-
-class Priority(TypedDict, total=False):
-    init: int
-    should_break: int
-    should_continue: int
-    before_run_iter: int
-    run_iter_context: int
-    after_run_iter: int
-    should_break_epoch: int
-    should_continue_epoch: int
-    before_run_epoch: int
-    run_epoch_context: int
-    after_run_epoch: int
-    before_run: int
-    after_run: int
-
-
-CallbackNames = Literal['init', 'should_break', 'should_continue',
-                        'befor

### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-02-08 02:30:01,373 88602:140704380690048][base.py:55 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:01,394 88602:140704380690048][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([10,  6]), 'y': tensor([20, 12])} weight=0.000 loss=16.000
[2024-02-08 02:30:01,396 88602:140704380690048][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([9, 2]), 'y': tensor([18,  4])} weight=0.040 loss=10.780
[2024-02-08 02:30:01,398 88602:140704380690048][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.067 loss=9.663
[2024-02-08 02:30:01,403 88602:140704380690048][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([5, 8]), 'y': tensor([10, 16])} weight=0.092 loss=12.399
[2024-02-0

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:30:01,431 88602:140704380690048][base.py:55 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:01,438 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([7, 4]), 'y': tensor([14,  8])} weight=0.000 loss=11.000 lr=['1.667e-03']
[2024-02-08 02:30:01,446 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([1, 6]), 'y': tensor([ 2, 12])} weight=0.009 loss=6.968 lr=['2.333e-03']
[2024-02-08 02:30:01,448 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([5, 9]), 'y': tensor([10, 18])} weight=0.017 loss=13.879 lr=['3.000e-03']
[2024-02-08 02:30:01,450 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([3,

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=4),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:30:01,493 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:01,494 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-02-08 02:30:01,497 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([4, 3]), 'y': tensor([8, 6])} weight=0.000 loss=7.000 lr=['1.667e-03']
[2024-02-08 02:30:01,500 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/10] batch={'x': tensor([1, 2]), 'y': tensor([2, 4])} weight=0.006 loss=2.991 lr=['1.667e-03']
[2024-02-08 02:30:01,502 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [2/5]
[2024-02-08 02:30:01,507 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-08 02:30:01,569 88602:140704380690048][lr.py:93 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-02-08 02:30:01,570 88602:140704380690048][base.py:55 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:01,574 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([8, 6]), 'y': tensor([16, 12])} weight=0.000 loss=14.000
[2024-02-08 02:30:01,576 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([ 3, 10]), 'y': tensor([ 6, 20])} weight=0.070 loss=12.545
[2024-02-08 02:30:01,578 88602:140704380690048][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([1, 5]), 'y': tensor([ 2, 10])} weight=0.135 loss=5.595
[2024-02-08 02:30:01,580 88602:140704380690048][log.py:9

### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-02-08 02:30:01,617 88602:140704380690048][base.py:55 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:01,623 88602:140704380690048][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 6]), 'y': tensor([ 6, 12])} weight=0.000 loss=9.000
[2024-02-08 02:30:01,625 88602:140704380690048][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpubd4uemq/checkpoint_callback/checkpoints/iter_1
[2024-02-08 02:30:01,633 88602:140704380690048][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([1, 9]), 'y': tensor([ 2, 18])} weight=0.022 loss=9.888
[2024-02-08 02:30:01,636 88602:140704380690048][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp

/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpubd4uemq
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opti

[2024-02-08 02:30:02,164 88602:140704380690048][checkpoint.py:54 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpubd4uemq/checkpoint_callback/checkpoints/iter_5
[2024-02-08 02:30:02,169 88602:140704380690048][base.py:82 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-08 02:30:02,170 88602:140704380690048][base.py:55 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:02,174 88602:140704380690048][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([10,  8]), 'y': tensor([20, 16])} weight=0.138 loss=16.763
[2024-02-08 02:30:02,176 88602:140704380690048][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpubd4uemq/checkpoint_callback/checkpoint

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-02-08 02:30:02,241 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:02,242 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-08 02:30:02,245 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([9, 8]), 'y': tensor([18, 16])} weight=0.000 loss=17.000
[2024-02-08 02:30:02,248 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([1, 5]), 'y': tensor([ 2, 10])} weight=0.043 loss=5.873
[2024-02-08 02:30:02,249 88602:140704380690048][checkpoint.py:80 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_wqnky9z/checkpoint_callback/checkpoints/iter_2
[2024-02-08 02:30:02,254 88602:14070


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_wqnky9z
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2024-02-08 02:30:02,794 88602:140704380690048][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_wqnky9z/checkpoint_callback/checkpoints/iter_8
[2024-02-08 02:30:02,798 88602:140704380690048][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-08 02:30:02,799 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:02,801 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2024-02-08 02:30:02,806 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor([3, 1]), 'y': tensor([6, 2])} weight=0.242 loss=3.515
[2024-02-08 02:30:02,809 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_cal


--------------------



[2024-02-08 02:30:03,274 88602:140704380690048][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_wqnky9z/checkpoint_callback/checkpoints/iter_10
[2024-02-08 02:30:03,278 88602:140704380690048][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-08 02:30:03,279 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:03,280 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-08 02:30:03,284 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([8, 4]), 'y': tensor([16,  8])} weight=0.275 loss=10.350
[2024-02-08 02:30:03,287 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoin

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-02-08 02:30:03,338 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:03,340 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-08 02:30:03,343 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([1, 5]), 'y': tensor([ 2, 10])} weight=0.000 loss=6.000
[2024-02-08 02:30:03,346 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.015 loss=9.925
[2024-02-08 02:30:03,349 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [3/15] batch={'x': tensor([9, 8]), 'y': tensor([18, 16])} weight=0.040 loss=16.660
[2024-02-08 02:30:03,352 88602:140704380690048][log.py:93 todd.Epoch


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpb9u7duhc
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> epoch_3

7 directories, 15 files



[2024-02-08 02:30:03,851 88602:140704380690048][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpb9u7duhc/checkpoint_callback/checkpoints/epoch_2
[2024-02-08 02:30:03,857 88602:140704380690048][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-08 02:30:03,859 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:03,865 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-08 02:30:03,872 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([7, 4]), 'y': tensor([14,  8])} weight=0.275 loss=9.488
[2024-02-08 02:30:03,875 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.checkpoint

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-02-08 02:30:04,052 88602:140704380690048][base.py:55 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:04,055 88602:140704380690048][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x14e916c50>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 223, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_88602/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2024-02-08 02:30:04,052 88602:140704380690048][base.py:55 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:04,055 88602:140704380690048][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x14e916c50>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 223, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_88602/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-02-08 02:30:04,397 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:04,398 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-08 02:30:04,405 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([7, 8]), 'y': tensor([14, 16])} weight=0.000 loss=15.000
[2024-02-08 02:30:04,409 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([6, 9]), 'y': tensor([12, 18])} weight=0.037 loss=14.719
[2024-02-08 02:30:04,412 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [3/15] batch={'x': tensor([3, 2]), 'y': tensor([6, 4])} weight=0.075 loss=4.812
[2024-02-08 02:30:04,416 88602:1407043806900


--------------------



[2024-02-08 02:30:04,958 88602:140704380690048][base.py:55 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-08 02:30:04,959 88602:140704380690048][base.py:97 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpk2owwwsa/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-02-08 02:30:04,963 88602:140704380690048][base.py:82 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-08 02:30:04,965 88602:140704380690048][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-08 02:30:04,969 88602:140704380690048][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([9, 2]), 'y': tensor([18,  4])} weight=0.275 loss=9.488
[2024-02-08 02:30:04,972 88602:1407043806

## Dry Run

In [28]:
todd.Store.DRY_RUN = True